In [5]:

import numpy as np
import pandas as pd
from pathlib import Path
from math import ceil

# -----------------------------
# 0) Load & basic wrangling
# -----------------------------
DATA_PATH = Path("MMM_Takehome_Dataset.csv")
if not DATA_PATH.exists():
    alt = Path("MMM Takehome Dataset (1).csv")
    if alt.exists(): DATA_PATH = alt
df = pd.read_csv(DATA_PATH)

df = df.copy()
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date").reset_index(drop=True)

# Clean NAs
for c in df.columns:
    if c.endswith("_spend") or c.endswith("_impressions") or c.endswith("_impression"):
        df[c] = df[c].fillna(0.0)
df["subscriptions"] = df["subscriptions"].clip(lower=0)

# Controls: trend + weekly seasonality (Fourier K=3)
df["t"] = np.arange(len(df), dtype=float)
df["weekofyear"] = df["date"].dt.isocalendar().week.astype(int)
for k in range(1, 3+1):
    df[f"sin_woy_{k}"] = np.sin(2*np.pi*k*df["weekofyear"]/52.0)
    df[f"cos_woy_{k}"] = np.cos(2*np.pi*k*df["weekofyear"]/52.0)

REVENUE_PER_SUB = 100.0
df["revenue_per_subscription"] = REVENUE_PER_SUB

# -----------------------------
# 1) Discover & align channels
# (only keep channels with BOTH spend & impressions)
# -----------------------------
spend_cols = [c for c in df.columns if c.endswith("_spend")]
impr_cols  = [c for c in df.columns if c.endswith("_impressions") or c.endswith("_impression")]

def _base(col: str) -> str:
    return (col.replace("_spend","").replace("_impressions","").replace("_impression",""))

def _impr_col_for(ch: str):
    if f"{ch}_impressions" in df.columns: return f"{ch}_impressions"
    if f"{ch}_impression"  in df.columns: return f"{ch}_impression"
    return None

channels = sorted(set(_base(c) for c in spend_cols + impr_cols))
aligned_channels, aligned_spend_cols, aligned_impr_cols = [], [], []
for ch in channels:
    sc, ic = f"{ch}_spend", _impr_col_for(ch)
    if (sc in df.columns) and (ic is not None):
        aligned_channels.append(ch)
        aligned_spend_cols.append(sc)
        aligned_impr_cols.append(ic)

print("Aligned channels:", aligned_channels)

# CPM-like factor to convert $ -> impressions (robust median)
impr_per_dollar = {}
for ch, sc, ic in zip(aligned_channels, aligned_spend_cols, aligned_impr_cols):
    denom = df[sc].replace(0, np.nan)
    ratio = (df[ic] / denom).replace([np.inf, -np.inf], np.nan).dropna()
    impr_per_dollar[ch] = float(np.median(ratio)) if len(ratio) else 0.0

# -----------------------------
# 2) Feature engineering
#    - geometric adstock
#    - Hill saturation (concave)
# -----------------------------
def adstock_geometric(x: np.ndarray, decay: float) -> np.ndarray:
    """x: (T,), decay in (0,1)."""
    out = np.zeros_like(x, dtype=float)
    for t in range(len(x)):
        out[t] = x[t] + (out[t-1] * decay if t > 0 else 0.0)
    return out

def hill_saturation(x: np.ndarray, alpha: float, ec50: float) -> np.ndarray:
    """alpha>0 (shape), ec50>0 (half-saturation)."""
    x_pos = np.clip(x, 0, None)
    return np.power(x_pos, alpha) / (np.power(x_pos, alpha) + np.power(ec50, alpha))

# Build control matrix
control_cols = ["t","sin_woy_1","cos_woy_1","sin_woy_2","cos_woy_2","sin_woy_3","cos_woy_3"]
X_controls = df[control_cols].to_numpy()
y = df["subscriptions"].to_numpy()

# -----------------------------
# 3) Train/holdout split
# -----------------------------
H = min(12, max(4, len(df)//10))  # last ~12 weeks for holdout (at least 4)
train_idx = np.arange(0, len(df)-H)
test_idx  = np.arange(len(df)-H, len(df))

# -----------------------------
# 4) Hyperparameter search (tiny, fast)
#    decay (0.2..0.9), alpha (0.3..2.0), ridge lambda (1e-3..1e2)
#    ec50 tuned per-channel to a data percentile of adstocked exposure
# -----------------------------
rng = np.random.default_rng(42)
n_search = 60

def fit_ridge(X, y, lam):
    # closed-form ridge (add small jitter for safety)
    XT_X = X.T @ X
    p = XT_X.shape[0]
    beta = np.linalg.solve(XT_X + lam*np.eye(p), X.T @ y)
    return beta

def mse(a, b): 
    d = a - b
    return float(np.mean(d*d))

best = {"score": np.inf}

for _ in range(n_search):
    decay  = float(rng.uniform(0.2, 0.9))
    alpha  = float(rng.uniform(0.3, 2.0))
    lam    = 10.0 ** float(rng.uniform(-3, 2))

    # Build media features with this (decay, alpha)
    X_media = []
    for ch, ic in zip(aligned_channels, aligned_impr_cols):
        # adstock on impressions
        ad = adstock_geometric(df[ic].to_numpy().astype(float), decay)
        # ec50 = percentile of adstocked exposure (robust to scale)
        ec50 = np.percentile(ad[train_idx], 70.0) + 1e-6
        sat = hill_saturation(ad, alpha, ec50)
        X_media.append(sat)
    X_media = np.vstack(X_media).T if X_media else np.zeros((len(df), 0))

    # Design matrices (add intercept)
    X_full   = np.hstack([np.ones((len(df),1)), X_controls, X_media])
    X_tr, y_tr = X_full[train_idx], y[train_idx]
    X_te, y_te = X_full[test_idx],  y[test_idx]

    beta = fit_ridge(X_tr, y_tr, lam)
    y_hat_te = X_te @ beta
    score = mse(y_te, y_hat_te)

    if score < best["score"]:
        best = dict(score=score, decay=decay, alpha=alpha, lam=lam, beta=beta, ec50s=[])

        # keep per-channel ec50 used (for later reporting)
        ec50s = []
        for ch, ic in zip(aligned_channels, aligned_impr_cols):
            ad = adstock_geometric(df[ic].to_numpy().astype(float), decay)
            ec50s.append(np.percentile(ad[train_idx], 70.0) + 1e-6)
        best["ec50s"] = ec50s

print(f"Best holdout MSE={best['score']:.3f} | decay={best['decay']:.3f} alpha={best['alpha']:.3f} lam={best['lam']:.4f}")

# -----------------------------
# 5) Refit on ALL data with best hyperparams
# -----------------------------
decay_opt = best["decay"]; alpha_opt = best["alpha"]; lam_opt = best["lam"]; ec50s_opt = best["ec50s"]

X_media_opt = []
for (ch, ic, ec50) in zip(aligned_channels, aligned_impr_cols, ec50s_opt):
    ad = adstock_geometric(df[ic].to_numpy().astype(float), decay_opt)
    sat = hill_saturation(ad, alpha_opt, ec50)
    X_media_opt.append(sat)
X_media_opt = np.vstack(X_media_opt).T if X_media_opt else np.zeros((len(df), 0))

X_full_opt = np.hstack([np.ones((len(df),1)), X_controls, X_media_opt])
beta_opt = np.linalg.solve(X_full_opt.T @ X_full_opt + lam_opt*np.eye(X_full_opt.shape[1]), X_full_opt.T @ y)
y_hat = X_full_opt @ beta_opt

# -----------------------------
# 6) Contributions & ROI
# -----------------------------
# Coefficient mapping
p_intercept = 1
p_controls  = X_controls.shape[1]
p_media     = X_media_opt.shape[1]
coefs_intercept = beta_opt[0]
coefs_controls  = beta_opt[1:1+p_controls]
coefs_media     = beta_opt[1+p_controls:1+p_controls+p_media]

# Per-channel contributions (sum over time of media_feature * beta)
contribs = {}
for j, ch in enumerate(aligned_channels):
    channel_feature = X_media_opt[:, j]
    contribs[ch] = float(np.sum(channel_feature * coefs_media[j]))

# Turn subs->revenue and compute ROI
roi_rows = []
for j, ch in enumerate(aligned_channels):
    subs_from_channel = contribs[ch]
    revenue = subs_from_channel * REVENUE_PER_SUB
    spend   = float(df[f"{ch}_spend"].sum()) + 1e-9
    roi     = revenue / spend   # $/$
    mroi    = coefs_media[j] * REVENUE_PER_SUB / (np.mean(df[f"{ch}_spend"])+1e-9)  # crude local
    roi_rows.append({
        "channel": ch,
        "subs_contrib_total": subs_from_channel,
        "revenue_contrib_total": revenue,
        "spend_total": spend,
        "ROI_$per$": roi,
        "local_mROI_$per$": mroi
    })
roi_df = pd.DataFrame(roi_rows).sort_values("ROI_$per$", ascending=False).reset_index(drop=True)

# -----------------------------
# 7) Report tables
# -----------------------------
summary = pd.DataFrame({
    "metric": ["RMSE_train_holdout", "decay", "alpha", "lambda", "subscriptions_mean", "subscriptions_sum"],
    "value":  [np.sqrt(best["score"]), decay_opt, alpha_opt, lam_opt, float(np.mean(y)), float(np.sum(y))]
})

# contributions table
contrib_df = pd.DataFrame({
    "channel": aligned_channels,
    "coefficient": coefs_media,
    "total_contribution_subs": [contribs[ch] for ch in aligned_channels],
    "total_spend": [float(df[f"{ch}_spend"].sum()) for ch in aligned_channels]
}).sort_values("total_contribution_subs", ascending=False).reset_index(drop=True)

# Display compact results
print("\n=== Model summary ===")
print(summary.to_string(index=False))
print("\n=== Channel ROI (sorted) ===")
print(roi_df[["channel","ROI_$per$","revenue_contrib_total","spend_total"]].to_string(index=False))

# Save artifacts
summary.to_csv("mmm_summary.csv", index=False)
roi_df.to_csv("channel_roi.csv", index=False)
contrib_df.to_csv("channel_contributions.csv", index=False)
pd.DataFrame({"date": df["date"], "subs": y, "subs_hat": y_hat}).to_csv("model_fit_timeseries.csv", index=False)

print("\nWrote: mmm_summary.csv, channel_roi.csv, channel_contributions.csv, model_fit_timeseries.csv")

# -----------------------------
# 8) Simple budget optimization
#    Greedy allocation by marginal ROI using the fitted response.
#    We assume weekly steady-state (coarse) and use current CPM to map $->impr.
# -----------------------------
def project_subs_given_spend(multiplier_by_channel: dict):
    """Project total subs if each channel spend is scaled by a multiplier."""
    proj_features = []
    for j, ch in enumerate(aligned_channels):
        sc, ic = f"{ch}_spend", aligned_impr_cols[j]
        m = multiplier_by_channel.get(ch, 1.0)
        # scale spend weekly, map to impressions via median factor
        scale = m
        impr_scale = impr_per_dollar[ch] if impr_per_dollar[ch] > 0 else 0.0
        # Build scaled impressions series (proportional to spend scaling)
        base_spend = df[sc].to_numpy()
        base_impr  = df[ic].to_numpy()
        # favor using actual impressions scaling from spend ratio when spend>0
        spend_ratio = (base_spend * scale) / (base_spend + 1e-9)
        impr_scaled = base_impr * spend_ratio
        # adstock + saturation with fitted params
        ad = adstock_geometric(impr_scaled, decay_opt)
        sat = hill_saturation(ad, alpha_opt, ec50s_opt[j])
        proj_features.append(sat)
    proj_features = np.vstack(proj_features).T if proj_features else np.zeros((len(df),0))
    X_proj = np.hstack([np.ones((len(df),1)), X_controls, proj_features])
    y_proj = X_proj @ beta_opt
    return float(np.sum(y_proj)), y_proj

# greedy allocator
def optimize_budget(total_multiplier=1.0, steps=200):
    current_spend_total = float(sum(df[f"{ch}_spend"].sum() for ch in aligned_channels))
    target_spend = current_spend_total * total_multiplier
    step_size = max(target_spend / steps, 1e-6)  # $ quantum
    # start from current multipliers = 1.0
    multipliers = {ch: 1.0 for ch in aligned_channels}
    # compute marginal gains by adding step_size to one channel at a time
    for _ in range(steps):
        best_ch, best_gain = None, -1e18
        base_total, _ = project_subs_given_spend(multipliers)
        for ch in aligned_channels:
            # add tiny increment proportionally to that channel's share
            inc = step_size / (df[f"{ch}_spend"].sum() + 1e-9)
            trial = multipliers.copy(); trial[ch] = multipliers[ch] + inc
            trial_total, _ = project_subs_given_spend(trial)
            gain = trial_total - base_total
            if gain > best_gain:
                best_gain, best_ch = gain, ch
        # apply increment to the best channel
        multipliers[best_ch] = multipliers[best_ch] + (step_size / (df[f"{best_ch}_spend"].sum() + 1e-9))
    return multipliers

scenarios = {
    "Budget_90pct": 0.90,
    "Budget_100pct": 1.00,
    "Budget_110pct": 1.10,
    "Budget_120pct": 1.20,
}

opt_rows = []
for name, mult in scenarios.items():
    mults = optimize_budget(total_multiplier=mult, steps=120)
    total_subs, yp = project_subs_given_spend(mults)
    row = {"scenario": name, "budget_multiplier": mult, "projected_subscriptions_total": total_subs}
    # derive channel allocation %
    alloc = []
    for ch in aligned_channels:
        alloc.append({
            "scenario": name,
            "channel": ch,
            "multiplier": mults[ch],
            "spend_total_new": float(df[f"{ch}_spend"].sum()) * mults[ch]
        })
    alloc_df = pd.DataFrame(alloc)
    alloc_df["allocation_share"] = alloc_df["spend_total_new"] / alloc_df["spend_total_new"].sum()
    alloc_df.to_csv(f"optimization_{name}_allocation.csv", index=False)
    print(f"Scenario {name}: projected subs total = {total_subs:,.1f} (alloc written to optimization_{name}_allocation.csv)")
    opt_rows.append(row)

opt_df = pd.DataFrame(opt_rows).sort_values("budget_multiplier")
opt_df.to_csv("optimization_scenarios_summary.csv", index=False)

print("\nWrote: optimization_scenarios_summary.csv and per-scenario allocation CSVs.")
print("Done.")


Aligned channels: ['amazon', 'beehiiv', 'engine', 'google', 'liveintent', 'meta', 'moloco', 'roku', 'snapchat', 'tiktok']
Best holdout MSE=5624327.822 | decay=0.315 alpha=0.376 lam=0.1498

=== Model summary ===
            metric         value
RMSE_train_holdout   2371.566533
             decay      0.315225
             alpha      0.376348
            lambda      0.149791
subscriptions_mean  12387.702703
 subscriptions_sum 916690.000000

=== Channel ROI (sorted) ===
   channel  ROI_$per$  revenue_contrib_total  spend_total
liveintent  15.210604           1.213787e+07 7.979872e+05
    moloco  13.845291           1.166098e+07 8.422346e+05
    engine   3.571356           5.231170e+06 1.464757e+06
  snapchat   2.098478           9.112509e+06 4.342437e+06
   beehiiv   1.880473           6.171192e+05 3.281723e+05
    tiktok   0.401850           7.043170e+05 1.752685e+06
    google   0.347429           3.578017e+06 1.029854e+07
      roku   0.000000           0.000000e+00 1.000000e-09
      

In [7]:
# ===== Meridian MMM — one-cell, end-to-end =====
# This cell:
# - Ensures google-meridian and a compatible TF/TFP are available in THIS kernel
# - Loads "MMM_Takehome_Dataset.csv"
# - Builds a single-geo Meridian InputData
# - Fits a Bayesian MMM (NUTS/TFP)
# - Exports summary + budget optimization + saved model

import sys, os, platform, subprocess, site
from pathlib import Path

def _pip(*args):
    cmd = [sys.executable, "-m", "pip"] + list(args)
    print("$", " ".join(cmd))
    out = subprocess.run(cmd, text=True)
    if out.returncode != 0:
        raise RuntimeError(f"pip failed: {' '.join(args)}")

# --- 0) Minimal dependency bootstrap (only if needed) ---
# We try a small matrix known to work with Meridian 1.2.x
# Strategy: pin NumPy<2, then try TFP/TF (2.16/0.24; 2.17/0.25), pick the first that works.

def _ensure_stack():
    pyver = f"{sys.version_info.major}.{sys.version_info.minor}"
    osys = platform.system().lower()
    arch = platform.machine().lower()
    print(f"Python {pyver} | OS {osys} | Arch {arch}")

    try:
        import numpy as _np  # noqa: F401
    except Exception:
        _pip("install", "-U", "numpy>=1.26,<2")

    # Meridian
    try:
        import meridian  # noqa: F401
    except Exception:
        _pip("install", "-U", "google-meridian")

    # If TF/TFP already present, try to import TFP quickly.
    def _try_import_tfp():
        try:
            import tensorflow_probability as tfp  # noqa: F401
            return True
        except Exception:
            return False

    if _try_import_tfp():
        print("TFP already present.")
        return

    # Choose TF package name by platform
    # Linux/Windows: tensorflow-cpu. macOS (Apple Silicon): tensorflow-macos.
    tf_pkg = "tensorflow-cpu"
    if osys == "darwin":
        tf_pkg = "tensorflow-macos"

    # Try two combos from newest to older:
    combos = [
        ("tensorflow-probability==0.25.0", f"{tf_pkg}==2.17.0"),
        ("tensorflow-probability==0.24.0", f"{tf_pkg}==2.16.1"),
    ]

    # Some wheels may not exist for Python 3.12 on macOS ARM; catch and fall back to guidance.
    for tfp_pin, tf_pin in combos:
        try:
            _pip("install", "-U", "numpy>=1.26,<2", tfp_pin, tf_pin)
            import tensorflow_probability as tfp  # noqa: F401
            print(f"Using {tfp_pin} with {tf_pin}")
            return
        except Exception as e:
            print(f"Combo {tfp_pin} / {tf_pin} failed: {e}")

    # If we got here, current kernel cannot get a compatible TF/TFP wheel.
    raise SystemExit(
        "This Python/OS combo cannot install a compatible TensorFlow/TFP for Meridian in-place. "
        "Create a fresh conda env with Python 3.10 or 3.11, then: "
        "conda create -n meridian-mmm python=3.10 -y && conda activate meridian-mmm && "
        "pip install -U google-meridian tensorflow-cpu==2.16.1 tensorflow-probability==0.24.0"
    )

_ensure_stack()

# --- 1) Imports (now that stack exists) ---
import numpy as np
import pandas as pd
from meridian.data.data_frame_input_data_builder import DataFrameInputDataBuilder
from meridian.model.model import Meridian, save_mmm
from meridian.model.spec import ModelSpec
from meridian.model import prior_distribution
from meridian import constants
from meridian.analysis.summarizer import Summarizer
from meridian.analysis.visualizer import ModelDiagnostics, ModelFit
from meridian.analysis.optimizer import BudgetOptimizer, FixedBudgetScenario
import tensorflow_probability as tfp

# --- 2) Load data ---
DATA_PATH = Path("MMM_Takehome_Dataset.csv")
if not DATA_PATH.exists():
    alt = Path("MMM Takehome Dataset (1).csv")
    if alt.exists():
        DATA_PATH = alt
if not DATA_PATH.exists():
    raise FileNotFoundError("Place MMM_Takehome_Dataset.csv next to this notebook.")

df = pd.read_csv(DATA_PATH)
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date").reset_index(drop=True)

# --- 3) Clean & controls ---
for c in df.columns:
    if c.endswith("_spend") or c.endswith("_impressions") or c.endswith("_impression"):
        df[c] = df[c].fillna(0.0)
df["subscriptions"] = df["subscriptions"].clip(lower=0)

# Trend + weekly Fourier seasonality (K=3)
df["t"] = np.arange(len(df), dtype=float)
df["weekofyear"] = df["date"].dt.isocalendar().week.astype(int)
for k in range(1, 4):
    df[f"sin_woy_{k}"] = np.sin(2*np.pi*k*df["weekofyear"]/52.0)
    df[f"cos_woy_{k}"] = np.cos(2*np.pi*k*df["weekofyear"]/52.0)

# Revenue per KPI for ROI interpretation
df["revenue_per_subscription"] = 100.0

# --- 4) Discover channels & ALIGN (require both spend + impressions) ---
spend_cols = [c for c in df.columns if c.endswith("_spend")]
impr_cols  = [c for c in df.columns if c.endswith("_impressions") or c.endswith("_impression")]

def _base(col: str) -> str:
    return col.replace("_spend","").replace("_impressions","").replace("_impression","")

def _impr_col_for(ch: str):
    if f"{ch}_impressions" in df.columns: return f"{ch}_impressions"
    if f"{ch}_impression"  in df.columns: return f"{ch}_impression"
    return None

channels = sorted(set(_base(c) for c in spend_cols + impr_cols))
media_channels, media_spend_cols, media_cols = [], [], []
for ch in channels:
    sc, ic = f"{ch}_spend", _impr_col_for(ch)
    if (sc in df.columns) and (ic is not None):
        media_channels.append(ch); media_spend_cols.append(sc); media_cols.append(ic)

control_cols = ["t","sin_woy_1","cos_woy_1","sin_woy_2","cos_woy_2","sin_woy_3","cos_woy_3"]
print("Aligned channels:", media_channels)

# --- 5) Build InputData (single-geo national model) ---
df["geo"] = "national"
builder = DataFrameInputDataBuilder(
    kpi_type="non_revenue",
    default_kpi_column="subscriptions",
    default_revenue_per_kpi_column="revenue_per_subscription",
    default_time_column="date",
    default_geo_column="geo",
)
builder = builder.with_kpi(df)
builder = builder.with_revenue_per_kpi(df)
builder = builder.with_controls(df, control_cols=control_cols)
builder = builder.with_media(
    df,
    media_cols=media_cols,
    media_spend_cols=media_spend_cols,
    media_channels=media_channels,
)
data = builder.build()
print("✔ Built Meridian InputData")

# --- 6) Model spec (weakly-informative ROI prior) ---
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(0.2, 0.9, name=constants.ROI_M)
)
model_spec = ModelSpec(prior=prior)

# --- 7) Fit Bayesian MMM ---
mmm = Meridian(input_data=data, model_spec=model_spec)
mmm.sample_prior(n_draws=200, seed=42)
mmm.sample_posterior(n_chains=4, n_adapt=1000, n_burnin=300, n_keep=800, seed=42)
print("✔ Sampling complete")

# --- 8) Diagnostics & fit plots ---
ModelDiagnostics(mmm).plot_rhat_boxplot()
ModelFit(mmm).plot_model_fit()

# --- 9) Two-page summary ---
sumr = Summarizer(mmm)
start_date = str(df["date"].min().date())
end_date   = str(df["date"].max().date())
sumr.output_model_results_summary("summary_output.html", ".", start_date, end_date)
print("✔ Wrote summary_output.html")

# --- 10) Budget optimization (fixed total budgets) ---
opt = BudgetOptimizer(mmm)
total_hist_budget = float(opt.get_current_media_spend().sum())
scenarios = [
    FixedBudgetScenario(total_budget=total_hist_budget * 0.90),
    FixedBudgetScenario(total_budget=total_hist_budget * 1.00),
    FixedBudgetScenario(total_budget=total_hist_budget * 1.10),
    FixedBudgetScenario(total_budget=total_hist_budget * 1.20),
]
opt_results = opt.optimize(scenarios=scenarios)
sumr.output_optimization_results_summary(
    "optimization_output.html", ".", start_date, end_date, optimization_results=opt_results
)
print("✔ Wrote optimization_output.html")

# --- 11) Save model ---
save_mmm(mmm, "saved_mmm.pkl")
print("✔ Saved model to saved_mmm.pkl")


Python 3.12 | OS darwin | Arch arm64
$ /opt/anaconda3/bin/python -m pip install -U google-meridian
TFP already present.


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject